# Validate, Clean and Transform

## Import libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

## Extract data from SQL server

In [ ]:
# Extract the source data from the bronze layer in the data warehouse
server = 'mohamedibrahim'
database = 'railway_dwh'

connection_string = (
    f'mssql+pyodbc://mohamedibrahim/railway_dwh'
    '?driver=ODBC+Driver+17+for+SQL+Server'
    '&Trusted_Connection=yes'
)
engine = create_engine(connection_string)

query = "SELECT * FROM bronze.railway"
df = pd.read_sql(query, engine)

## Data Overview

In [3]:
df.head()

,TransactionID,Date_of_Purchase,Time_of_Purchase,Purchase_Type,Payment_Method,Railcard,Ticket_Class,Ticket_Type,Price,Departure_Station,Arrival_Station,Date_of_Journey,Departure_Time,Arrival_Time,Actual_Arrival_Time,Journey_Status,Reason_for_Delay,Refund_Request
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01,11:00:00,13:30:00,13:30:00,On Time,None,No
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,None,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02,18:15:00,18:45:00,18:45:00,On Time,None,No
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,None,Standard,Advance,13,London Paddington,Reading,2024-01-01,21:30:00,22:30:00,22:30:00,On Time,None,No
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,None,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01,16:45:00,19:00:00,19:00:00,On Time,None,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31653 entries, 0 to 31652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   TransactionID        31653 non-null  object
 1   Date_of_Purchase     31653 non-null  object
 2   Time_of_Purchase     31653 non-null  object
 3   Purchase_Type        31653 non-null  object
 4   Payment_Method       31653 non-null  object
 5   Railcard             31653 non-null  object
 6   Ticket_Class         31653 non-null  object
 7   Ticket_Type          31653 non-null  object
 8   Price                31653 non-null  int64 
 9   Departure_Station    31653 non-null  object
 10  Arrival_Station      31653 non-null  object
 11  Date_of_Journey      31653 non-null  object
 12  Departure_Time       31653 non-null  object
 13  Arrival_Time         31653 non-null  object
 14  Actual_Arrival_Time  29773 non-null  object
 15  Journey_Status       31653 non-null  object
 16  Reas

In [5]:
df.isnull().sum()

TransactionID              0
Date_of_Purchase           0
Time_of_Purchase           0
Purchase_Type              0
Payment_Method             0
Railcard                   0
Ticket_Class               0
Ticket_Type                0
Price                      0
Departure_Station          0
Arrival_Station            0
Date_of_Journey            0
Departure_Time             0
Arrival_Time               0
Actual_Arrival_Time     1880
Journey_Status             0
Reason_for_Delay       27481
Refund_Request             0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(0)

In [10]:
# Checks each object column for unwanted spaces (leading spaces, trailing spaces, multiple spaces)
for col in df.select_dtypes(include='object').columns:
    has_unwanted_spaces = False
    
    # Iterate through non-null values and check each one
    for value in df[col][df[col].notna()]:
        # Convert to string and check for unwanted spaces
        str_value = str(value)
        # Check for leading/trailing spaces or multiple consecutive spaces
        if str_value != str_value.strip() or '  ' in str_value:
            has_unwanted_spaces = True
            break
    
    if has_unwanted_spaces:
        print(f"Unwanted spaces found in column: {col}")

    else:
        print("It's all good :)")   

It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)
It's all good :)


## Naming Convention

### This section outlines the naming conventions used for tables, columns and other objects in the data warehouse.

#### Naming Conventions: Use snake_case, with lowercase letters and underscores (_) to separate words, this applies to any tables, columns and objects exepect columns values, stays as it is.

In [7]:
df.columns = df.columns.str.lower()

In [8]:
df = df.rename(columns={'transactionid': 'transaction_id'})

## Data Inspection & Cleaning

#### Column "transaction_id"

In [10]:
df['transaction_id'].is_unique

True

#### Column "date_of_purchase"

In [11]:
df['date_of_purchase'] = pd.to_datetime(df['date_of_purchase'], errors='coerce')

In [12]:
df['date_of_purchase'].isna().sum()

np.int64(0)

In [13]:
# check for illogical purchase dates
df.loc[df['date_of_purchase'] > df['date_of_journey']]

,transaction_id,date_of_purchase,time_of_purchase,purchase_type,payment_method,railcard,ticket_class,ticket_type,price,departure_station,arrival_station,date_of_journey,departure_time,arrival_time,actual_arrival_time,journey_status,reason_for_delay,refund_request


#### Column "time_of_purchase"

In [14]:
df['time_of_purchase'] = pd.to_datetime(df['time_of_purchase'], format='%H:%M:%S', errors='coerce').dt.time

In [15]:
df['time_of_purchase'].isna().sum()

np.int64(0)

In [16]:
# check for illogical purchase time
df.loc[(df['date_of_purchase'] == df['date_of_journey']) & (df['time_of_purchase'] > df['departure_time'])]

,transaction_id,date_of_purchase,time_of_purchase,purchase_type,payment_method,railcard,ticket_class,ticket_type,price,departure_station,arrival_station,date_of_journey,departure_time,arrival_time,actual_arrival_time,journey_status,reason_for_delay,refund_request


#### Column "purchase_type"

In [17]:
df['purchase_type'].unique()

array(['Online', 'Station'], dtype=object)

#### Column "payment_method"

In [18]:
df['payment_method'].unique()

array(['Contactless', 'Credit Card', 'Debit Card'], dtype=object)

#### Column "railcard"

In [19]:
df['railcard'].unique()

array(['Adult', 'None', 'Disabled', 'Senior'], dtype=object)

#### Column "ticket_class"

In [20]:
df['ticket_class'].unique()

array(['Standard', 'First Class'], dtype=object)

#### Column "ticket_type"

In [21]:
df['ticket_type'].unique()

array(['Advance', 'Off-Peak', 'Anytime'], dtype=object)

#### Column "price"

In [22]:
np.sort(df['price'].unique())

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        41,  42,  43,  44,  45,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  59,  63,  64,  65,  66,  67,  69,  70,  71,  72,
        73,  74,  76,  77,  78,  80,  82,  84,  86,  88,  89,  90,  93,
        94,  95,  96,  97, 101, 102, 104, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 116, 117, 118, 119, 121, 126, 128, 129, 134, 135,
       143, 144, 146, 151, 154, 157, 158, 162, 168, 171, 176, 178, 180,
       200, 203, 211, 216, 235, 238, 242, 267])

#### Column "departure_station"

In [23]:
df['departure_station'].unique()

array(['London Paddington', 'London Kings Cross', 'Liverpool Lime Street',
       'London Euston', 'York', 'Manchester Piccadilly',
       'Birmingham New Street', 'London St Pancras', 'Oxford', 'Reading',
       'Edinburgh Waverley', 'Bristol Temple Meads'], dtype=object)

#### Column "arrival_station"

In [24]:
df['arrival_station'].unique()

array(['Liverpool Lime Street', 'York', 'Manchester Piccadilly',
       'Reading', 'London Euston', 'Oxford', 'Durham',
       'London St Pancras', 'Birmingham New Street', 'London Paddington',
       'Bristol Temple Meads', 'Tamworth', 'London Waterloo', 'Sheffield',
       'Wolverhampton', 'Leeds', 'Stafford', 'Doncaster', 'Swindon',
       'Nottingham', 'Peterborough', 'Edinburgh', 'Crewe',
       'London Kings Cross', 'Leicester', 'Nuneaton', 'Didcot',
       'Edinburgh Waverley', 'Coventry', 'Wakefield', 'Cardiff Central',
       'Warrington'], dtype=object)

#### Column "date_of_journey"

In [25]:
df['date_of_journey'] = pd.to_datetime(df['date_of_journey'], errors='coerce')

In [26]:
df['date_of_journey'].isnull().sum()

np.int64(0)

#### Column "departure_time"

In [27]:
df['departure_time'] = pd.to_datetime(df['departure_time'], format='%H:%M:%S', errors='coerce').dt.time

In [28]:
df['departure_time'].isnull().sum()

np.int64(0)

#### Column "arrival_time"

In [29]:
df['arrival_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M:%S', errors='coerce').dt.time

In [30]:
df['arrival_time'].isnull().sum()

np.int64(0)

#### Column "actual_arrival_time"

In [31]:
df['actual_arrival_time'] = pd.to_datetime(df['actual_arrival_time'], format='%H:%M:%S', errors='coerce').dt.time

In [32]:
df['actual_arrival_time'].isnull().sum()

np.int64(1880)

#### The nulls in the column "actual_arrival_time" represent the cancelled journeys in the column "journey_status"

In [33]:
cancelled_journeys = df[(df['journey_status'] == 'Cancelled') & (df['actual_arrival_time'].isnull())]
print(cancelled_journeys)

                transaction_id date_of_purchase time_of_purchase  \
39     69eaff73-ca3a-4831-905b       2024-01-01         03:52:29   
75     a88f097f-bda3-4394-9c7d       2024-01-01         17:43:02   
91     8a66ead7-e381-4311-b667       2024-01-02         00:54:33   
111    d618c56f-3512-484d-9876       2024-01-02         03:33:32   
148    eace2ada-5294-47e6-b4b8       2024-01-02         06:33:00   
...                        ...              ...              ...   
31639  465e3643-fb67-4deb-8ec9       2024-04-30         17:13:32   
31641  2e96cde6-50c1-4311-8089       2024-04-30         17:16:04   
31642  09e207d9-db49-4ec0-9c4f       2024-04-30         17:21:14   
31644  1eacd955-f539-4c79-adb6       2024-04-30         17:26:33   
31645  a0e238b7-1ad1-4bf7-bd6c       2024-04-30         18:11:50   

      purchase_type payment_method  railcard ticket_class ticket_type  price  \
39           Online    Credit Card      None     Standard     Advance      7   
75          Station    

In [34]:
# If the arrival time equals the actual arrival time then the journey status must be on time and the reason for delay must be no delay
not_ontime_journeys = df[((df['arrival_time']) == (df['actual_arrival_time'])) & (df['journey_status'] != 'On Time')]
print(not_ontime_journeys)
# There is 18 rows that the journey status is delayed and there is reasons for delayes in this case, and that is not right

                transaction_id date_of_purchase time_of_purchase  \
10633  f10dc9f2-80c3-4b9f-8b72       2024-02-06         05:01:05   
13933  add29bde-e183-426a-adca       2024-02-15         15:01:47   
15130  3d6c240e-5c33-4665-9144       2024-02-21         11:54:54   
16274  2b2bf794-2111-44bf-8758       2024-03-03         10:45:53   
16483  bd082832-41f9-4364-a8d2       2024-03-04         07:46:54   
16488  73bc8893-5e5f-47c6-951b       2024-03-04         07:56:08   
16868  97203c12-be97-4199-8ac0       2024-03-05         16:11:29   
16879  3d6779a3-1206-4b3b-872f       2024-03-05         17:07:35   
18927  9fe75f16-a67a-4d45-9c92       2024-03-13         04:19:37   
22975  9479bec9-2e01-4aac-be28       2024-03-28         05:09:54   
23128  1923b77a-c469-41e7-98ea       2024-03-28         17:14:18   
25003  c6a831e2-45a2-4089-8161       2024-04-06         02:01:10   
25740  bfea5b54-7877-4ab1-9fed       2024-04-08         17:13:59   
27923  441924c9-c008-4102-8b1d       2024-04-16 

In [35]:
# Correcting the journey_status and the reason_for_delay
df.loc[((df['arrival_time']) == (df['actual_arrival_time'])) & (df['journey_status'] != 'On Time'), ['journey_status','reason_for_delay'] ] = ['On Time', 'No Delay']

#### Column "journey_status"

In [36]:
df['journey_status'].unique()

array(['On Time', 'Delayed', 'Cancelled'], dtype=object)

#### Column "reason_for_delay"

In [37]:
df['reason_for_delay'].unique()

array([None, 'Signal Failure', 'Technical Issue', 'Weather Conditions',
       'Weather', 'Staffing', 'Staff Shortage', 'Signal failure',
       'Traffic', 'No Delay'], dtype=object)

In [38]:
df['reason_for_delay'] = df['reason_for_delay'].replace({'Signal failure': 'Signal Failure',
                                                         'Weather': 'Weather Conditions',
                                                         'Staffing': 'Staffing Issues',
                                                         'Staff Shortage': 'Staffing Issues'})

In [39]:
df['reason_for_delay'] = df['reason_for_delay'].fillna('No Delay')

#### Column "refund_request"

In [40]:
df['refund_request'].unique()

array(['No', 'Yes'], dtype=object)

## Load Transformed Data into SQL Server

In [41]:
# Load the transformed "railway" data into the silver layer in the data warehouse

server = 'mohamedibrahim'
database = 'railway_dwh'

connection_string = (
    f'mssql+pyodbc://mohamedibrahim/railway_dwh'
    '?driver=ODBC+Driver+17+for+SQL+Server'
    '&Trusted_Connection=yes'
)

engine = create_engine(connection_string)

df.to_sql(
    name='railway',
    schema='silver',
    con=engine,   
    if_exists='append',
    index=False
)

101